## Song recommendation with colaborative filtering for recommendation

Imporing all the necessary libraries and loading the data

In [4]:
import numpy as np

songs = [
    "Luis Fonsi - Despacito ft. Daddy Yankee",
    "Gorillaz - Feel Good",
    "U2 - One",
    "Milky Chance - Stolen Dance",
    "Major Lazer & DJ Snake - Lean On",
    "Foo Fighters - Everlong",
    "Coldplay - Viva la Vida"
]

# Songs Column Wise: Luis Fonsi - Despacito ft. Daddy Yankee; Gorillaz - Feel Good; U2 - One; Milky Chance - Stolen Dance;
# Major Lazer & DJ Snake - Lean On; Foo Fighters - Everlong; Coldplay - Viva la Vida .
P = [
    [0,  0, -1,  0, -1,  1,  1], # User 1
    [-1, 1,  1, -1,  0,  1,  1], # User 2
    [0,  1,  1,  0,  0, -1,  1], # User 3
    [-1, 1,  1,  0,  0,  1,  1], # User 4
    [0,  1,  1,  0,  0,  1,  1], # User 5
    [1, -1,  1,  1,  1, -1,  0], # User 6
    [-1, 1, -1,  0, -1,  0,  1], # User 7
    [0, -1,  0,  1,  1, -1, -1], # User 8
    [0,  0, -1,  1,  1,  0, -1]  # User 9
]

In [5]:
P = np.array(P)

# Parameters
reg = 0.1 # regularization parameter
f = 2 # number of factors
m, n = P.shape # where m = 9 and n = 7


# Random Initialization with random features
# X is (m x f) 9 x 2
# Y is (f x n) 2 x 7
X = 1 - 2 * np.random.rand(m, f)
Y = 1 - 2 * np.random.rand(f, n)
X *= 0.1
Y *= 0.1


# Alternating Ridge Regression
for _ in range(100):
    # Least-squares keeping Y fixed
    
    X = np.linalg.solve(
        np.dot(Y, Y.T) + reg * np.eye(f),
        np.dot(Y, P.T)
    ).T
    
    # Least-squares keeping X fixed
    Y = np.linalg.solve(
        np.dot(X.T, X) + reg * np.eye(f),
        np.dot(X.T, P)
    )
    
print("Alternating Ridge Regression:")
print(np.dot(X, Y))
print("\n")

Alternating Ridge Regression:
[[-0.47047886  0.42881318 -0.92322659 -0.40431643 -0.82818466  0.71377061
   0.3504229 ]
 [-0.57771675  1.11906096  0.94589673 -0.61385994 -0.41293439  0.72761533
   1.24992228]
 [-0.08329322  0.47515054  1.23776785 -0.15067525  0.26037107  0.02607128
   0.61430642]
 [-0.47973268  0.97615727  0.95005909 -0.51903661 -0.29509155  0.59242669
   1.1027997 ]
 [-0.37593978  0.83720216  0.99806081 -0.42105254 -0.15760084  0.44610341
   0.96413657]
 [ 0.6725478  -0.68837676  1.05514853  0.59290494  1.10703196 -1.00139328
  -0.60521655]
 [-0.56690357  0.63195511 -0.70791891 -0.51001554 -0.88042481  0.83110365
   0.58167881]
 [ 0.6594163  -0.96038537  0.03268837  0.63788108  0.79442075 -0.91013153
  -0.98826706]
 [ 0.32613618 -0.6776028  -0.69495396  0.35562638  0.18635727 -0.39923613
  -0.76905787]]




In [6]:
# Re-initialize
X = 1 - 2 * np.random.rand(m, f)
Y = 1 - 2 * np.random.rand(f, n)
X *= 0.1
Y *= 0.1


# Alternating Weighted Ridge Regression
C = np.abs(P) # Will be 0 only when P[i,j] == 0.

for _ in range(100):
    
    # Each user u has a different set of weights Cu
    for u, Cu in enumerate(C):
        idCu = np.diag(Cu)
        
#         X[u] = np.dot(
#             np.linalg.inv(np.dot(np.dot(Y, idCu), Y.T) + np.eye(f) * reg),
#             np.dot(np.dot(Y, idCu), Cu)
#         )
        
        X[u] = np.linalg.solve(
            np.dot(np.dot(Y, idCu), Y.T) +  np.eye(f) * reg,
            np.dot(np.dot(Y, idCu), P[u])
        ).T
    
        
    for i, Ci in enumerate(C.T):
        idCi = np.diag(Ci)
        
#         Y[:, i] = np.dot(
#             np.linalg.inv(np.dot(np.dot(X.T, idCi), X) + np.eye(f) * reg),
#             np.dot(np.dot(X.T, idCi), Ci)
#         )

        Y[:, i] = np.linalg.solve(
            np.dot(np.dot(X.T, idCi), X) + np.eye(f) * reg,
            np.dot(np.dot(X.T, idCi), P[:, i])
        )

Yhat = np.dot(X, Y)

print("Alternating Weighted Ridge Regression:")
print(Yhat)

Alternating Weighted Ridge Regression:
[[-0.87859008  0.75590705 -0.84723565 -0.90527023 -0.91301325  1.22456418
   0.77033648]
 [-0.96652361  1.1203742   0.83860296 -1.00625928 -1.00816692  0.72382425
   1.10749937]
 [-0.3254      0.55731417  1.38658514 -0.34525457 -0.34177429 -0.1450296
   0.5350507 ]
 [-0.9680409   1.12193574  0.83871    -1.00783186 -1.00974701  0.72538629
   1.10906034]
 [-0.95828412  1.11192617  0.83821628 -0.99772071 -0.99958685  0.71527327
   1.0990513 ]
 [ 0.92243181 -0.8026479   0.83420724  0.95076768  0.95869061 -1.26620134
  -0.81689941]
 [-1.02703803  0.89416352 -0.92578485 -1.058605   -1.06741512  1.40872753
   0.90998209]
 [ 0.95136665 -0.98077071 -0.07729396  0.98609104  0.99076188 -0.97584058
  -0.98024506]
 [ 0.8969281  -1.06911013 -0.95852122  0.93486009  0.93595724 -0.60823423
  -1.05423492]]


In [69]:
users_test = [0, 3, 4, 7, 8]

message = """User {}:\n
    Likes: {}\n
    Dislikes: {}\n\n
    Prediction Likes: {}\n
    Prediction Dislikes: {}\n
    \n\n"""

for user in users_test:
    dislikes = []
    likes = []

    predictions = [[], []]
    for content, y in enumerate(P[user]):
        if(y == -1):
            dislikes.append(songs[content])
        elif(y == 1):
            likes.append(songs[content])
        else:
            if(Yhat[user][content] > 0.0):
                predictions[0].append(songs[content])
            else: predictions[1].append(songs[content])
        
            
    print(message.format("user " + str(user + 1),\
                  "; ".join(likes),\
                  "; ".join(dislikes),\
                  "; ".join(predictions[0]),\
                 "; ".join(predictions[1])))

User user 1:

    Likes: Foo Fighters - Everlong; Coldplay - Viva la Vida

    Dislikes: U2 - One; Major Lazer & DJ Snake - Lean On


    Prediction Likes: Gorillaz - Feel Good

    Prediction Dislikes: Luis Fonsi - Despacito ft. Daddy Yankee; Milky Chance - Stolen Dance

    


User user 4:

    Likes: Gorillaz - Feel Good; U2 - One; Foo Fighters - Everlong; Coldplay - Viva la Vida

    Dislikes: Luis Fonsi - Despacito ft. Daddy Yankee


    Prediction Likes: 

    Prediction Dislikes: Milky Chance - Stolen Dance; Major Lazer & DJ Snake - Lean On

    


User user 5:

    Likes: Gorillaz - Feel Good; U2 - One; Foo Fighters - Everlong; Coldplay - Viva la Vida

    Dislikes: 


    Prediction Likes: 

    Prediction Dislikes: Luis Fonsi - Despacito ft. Daddy Yankee; Milky Chance - Stolen Dance; Major Lazer & DJ Snake - Lean On

    


User user 8:

    Likes: Milky Chance - Stolen Dance; Major Lazer & DJ Snake - Lean On

    Dislikes: Gorillaz - Feel Good; Foo Fighters - Everlong; Coldp